# 06_nbo_llm_demo.ipynb

Демо полного пайплайна NBO: ML-ранжирование офферов + LLM-сообщения.

In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd

from src.utils.config import ML_TRAINING_DATASET_PATH
from src.service.nbo_pipeline import get_nbo_response, get_nbo_response_from_rows
from src.ml.ranking_model import load_training_data

In [2]:
def response_to_dataframe(resp: dict) -> pd.DataFrame:
    """Преобразуем best + alternatives в таблицу."""
    if not resp or resp.get("best_offer") is None:
        return pd.DataFrame()

    rows = [resp["best_offer"], *resp.get("alternative_offers", [])]
    return pd.DataFrame(rows)

In [3]:
df_ml = load_training_data()
df_ml.head()

2025-12-07 01:25:00 | INFO | src.ml.ranking_model:load_training_data - Loaded ML training dataset: %s, shape=%s


,client_id,treatment,offer_id,treatment_date,offer_type,offer_category,cost,offer_AOV,channel,conversion,...,category_affinity_top1,is_mobile_user,city_tier,email_open_rate_30d,push_enabled,age,gender,price_segment,treatment_dow,treatment_month
0,17850,1,0,2024-04-12,discount_10,category_0,2.09,18.15,app,0,...,0.70,0,3,0.98,1,37,F,budget,4,4
1,13047,1,2,2024-03-12,free_delivery,category_8,5.65,18.82,app,0,...,0.77,1,3,0.76,1,58,M,budget,1,3
2,12583,1,1,2024-01-21,discount_20,category_8,5.13,29.48,web,0,...,0.48,1,3,0.25,1,40,F,budget,6,1
3,12583,1,0,2024-04-12,discount_10,category_8,7.16,29.48,app,1,...,0.45,1,3,0.71,1,35,F,budget,4,4
4,12583,1,0,2024-05-01,discount_10,category_8,1.49,29.48,web,0,...,0.91,0,3,0.14,1,65,F,budget,2,5


In [4]:
sample_client_id = int(df_ml["client_id"].sample(1, random_state=42).iloc[0])
sample_client_id

16189

In [5]:
channel = "push"
top_n = 3

resp_internal = get_nbo_response(
    client_id=sample_client_id,
    top_n=top_n,
    channel=channel,
    provider="gigachat"
)

print("CLIENT_ID:", resp_internal["client_id"])
print("CHANNEL:", resp_internal["channel"])
print("\nUSER PROFILE:\n")
print(resp_internal["user_profile"])

print("\nTOP-N OFFERS:\n")
response_to_dataframe(resp_internal)

CLIENT_ID: 16189
CHANNEL: push

USER PROFILE:

Краткий профиль клиента:

• Давность последней покупки: 15 дней.
• Частота покупок за 90 дней: 11.
• Сумма трат за 90 дней: 215.48.
• Интересуется категорией: category_8.
• Сегмент цены: budget.
• Использовал скидки: 4 раз за 90 дней.
• Открываемость e-mail писем: 0.81.

Общий вывод: сформировать аккуратное персонализированное сообщение с учётом интересов и сегмента.

TOP-N OFFERS:



,offer_id,p_click,title,short_description,conditions,personalized_message
0,0,0.332210,Offer 0,,,Подбирайте товары категории 8 с умом — наш офф...
1,1,0.282992,Offer 1,,,Для вас специальное предложение: 20% скидка на...


In [6]:
print(json.dumps(resp_internal, ensure_ascii=False, indent=2))

{
  "client_id": 16189,
  "channel": "push",
  "user_profile": "Краткий профиль клиента:\n\n• Давность последней покупки: 15 дней.\n• Частота покупок за 90 дней: 11.\n• Сумма трат за 90 дней: 215.48.\n• Интересуется категорией: category_8.\n• Сегмент цены: budget.\n• Использовал скидки: 4 раз за 90 дней.\n• Открываемость e-mail писем: 0.81.\n\nОбщий вывод: сформировать аккуратное персонализированное сообщение с учётом интересов и сегмента.",
  "best_offer": {
    "offer_id": 0,
    "p_click": 0.33220952256466924,
    "title": "Offer 0",
    "short_description": "",
    "conditions": "",
    "personalized_message": "Подбирайте товары категории 8 с умом — наш оффер даст вам 10% экономии!"
  },
  "alternative_offers": [
    {
      "offer_id": 1,
      "p_click": 0.28299229811218407,
      "title": "Offer 1",
      "short_description": "",
      "conditions": "",
      "personalized_message": "Для вас специальное предложение: 20% скидка на любимые товары категории, ваша экономия уже готов

### Сценарий 2: онлайн-вход (rows приходят снаружи)

In [7]:
client_rows = df_ml[df_ml["client_id"] == sample_client_id]

n_samples = min(5, len(client_rows))  # не больше реального числа строк

rows_for_client = (
    client_rows
    .sample(n_samples, random_state=42)
    .drop(columns=["conversion"])  # таргет снаружи не нужен
)

rows_for_client.head()

,client_id,treatment,offer_id,treatment_date,offer_type,offer_category,cost,offer_AOV,channel,recency_days,...,category_affinity_top1,is_mobile_user,city_tier,email_open_rate_30d,push_enabled,age,gender,price_segment,treatment_dow,treatment_month
9778,16189,1,0,2024-02-12,discount_10,category_8,2.23,19.59,app,15,...,0.52,1,3,0.81,1,55,M,budget,0,2
9777,16189,1,1,2024-04-16,discount_20,category_8,2.09,19.59,app,15,...,0.68,1,3,0.07,1,54,F,budget,1,4


In [8]:
rows_payload = rows_for_client.to_dict(orient="records")
len(rows_payload)

2

In [9]:
resp_online = get_nbo_response_from_rows(
    rows=rows_payload,
    client_id=sample_client_id,
    top_n=top_n,
    channel=channel,
    # provider="gigachat"  # при желании можно явно зафиксировать
)

print("ONLINE MODE — CLIENT_ID:", resp_online["client_id"])
print("CHANNEL:", resp_online["channel"])
print("\nUSER PROFILE:\n")
print(resp_online["user_profile"])

print("\nTOP-N OFFERS (online rows):\n")
response_to_dataframe(resp_online)

ONLINE MODE — CLIENT_ID: 16189
CHANNEL: push

USER PROFILE:

Краткий профиль клиента:

• Давность последней покупки: 15 дней.
• Частота покупок за 90 дней: 11.
• Сумма трат за 90 дней: 215.48.
• Интересуется категорией: category_8.
• Сегмент цены: budget.
• Использовал скидки: 4 раз за 90 дней.
• Открываемость e-mail писем: 0.81.

Общий вывод: сформировать аккуратное персонализированное сообщение с учётом интересов и сегмента.

TOP-N OFFERS (online rows):



,offer_id,p_click,title,short_description,conditions,personalized_message
0,0,0.332210,Offer 0,,,Тестовое сообщение: здесь будет персонализиров...
1,1,0.282992,Offer 1,,,Тестовое сообщение: здесь будет персонализиров...


In [10]:
df_int = response_to_dataframe(resp_internal).assign(source="internal")
df_onl = response_to_dataframe(resp_online).assign(source="online")

pd.concat([df_int, df_onl], ignore_index=True)

,offer_id,p_click,title,short_description,conditions,personalized_message,source
0,0,0.332210,Offer 0,,,Подбирайте товары категории 8 с умом — наш офф...,internal
1,1,0.282992,Offer 1,,,Для вас специальное предложение: 20% скидка на...,internal
2,0,0.332210,Offer 0,,,Тестовое сообщение: здесь будет персонализиров...,online
3,1,0.282992,Offer 1,,,Тестовое сообщение: здесь будет персонализиров...,online
